# App Response time per application per source (P1)

Here we testing the different models for the p1 report with latest architecture (aggregated data to one day)

## Data preperation

In [1]:

from pyspark.sql import SQLContext
import sys
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession


from pyspark.sql import SparkSession
import pymysql
from tqdm import tqdm
import config

# initialise sparkContext
spark1 = SparkSession.builder \
        .master(config.sp_master) \
        .appName(config.sp_appname) \
        .config('spark.executor.memory', config.sp_memory) \
        .config("spark.cores.max", config.sp_cores) \
        .getOrCreate()
    
sc = spark1.sparkContext

# using SQLContext to read parquet file
sqlContext = SQLContext(sc)

In [2]:
df = sqlContext.read.parquet('./../datas/appid_datapoint_parquet1')

In [3]:
ap_list = ['DNS', 'DHCP', 'Radius', 'LDAP','Kerberos']
s_list  = ['10.6.1.101','134.141.5.104']

# Prophet model

In [4]:
import pandas as pd
import pymysql
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import logging
from joblib import Parallel, delayed
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error as mse
import math


def connect_to_mysql():
    connection = pymysql.connect(host = config.db_host,
                            port= config.db_port,
                            user= config.db_user,
                            password= config.db_pass,
                            db= config.db_name,
                            charset='utf8',
                            cursorclass=pymysql.cursors.DictCursor)
    return connection


### Prophet model train and forecast funtion

In [6]:
def create_prophet_m(app_name,z1,delay=24):
    
    ### --- For realtime pred ---###
    
    full_df = z1.app_rsp_time.iloc[0:len(z1)]
    full_df = full_df.reset_index()
    full_df.columns = ['ds','y']
    
    #removing outliers
    q50 = full_df.y.median()
    q100 = full_df.y.quantile(1)
    q75  = full_df.y.quantile(.75)
    
    #if((q100-q50) >= (2*q75)):
        
    #    full_df.loc[full_df.y>=(2*q75),'y'] = None
    
    if(len(full_df.dropna())>=10):
        
        #-- Realtime prediction --##
        #model 
        model_r = Prophet(yearly_seasonality=False,changepoint_prior_scale=.1,seasonality_prior_scale=0.05)
        model_r.fit(full_df)
        future_r = model_r.make_future_dataframe(periods=delay,freq='D')
        forecast_r = model_r.predict(future_r)
        forecast_r.index = forecast_r['ds']
        #forecast 
        pred_r = pd.DataFrame(forecast_r['yhat'][len(z1):(len(z1)+delay)])
        pred_r=pred_r.reset_index()
        #--- completes realtime pred ---#

    #----- validation ----#    
    train_end_index=len(z1.app_rsp_time)-delay
    train_df=z1.app_rsp_time.iloc[0:train_end_index]
    
    test_df=z1.app_rsp_time.iloc[train_end_index:len(z1)]
    
    train_df=train_df.reset_index()
    test_df=test_df.reset_index()
    train_df.columns=['ds','y']
    
    #--- removing outliers in trainset  ---#
    
    q50 = train_df.y.median()
    q100 = train_df.y.quantile(1)
    q75  = train_df.y.quantile(.75)
    
    if((q100-q50) >= (2*q75)):
        
        train_df.loc[train_df.y>=(2*q75),'y'] = None
        
    if(len(train_df.dropna())>=10):
    
        test_df.columns=['ds','y']
        test_df['ds'] = pd.to_datetime(test_df['ds'])
        
        #model 
        model = Prophet(yearly_seasonality=False,changepoint_prior_scale=.1,seasonality_prior_scale=0.05)
        model.fit(train_df)
    
        future = model.make_future_dataframe(periods=len(test_df),freq='D')
        forecast = model.predict(future)
        forecast.index = forecast['ds']
        #forecast 
        pred = pd.DataFrame(forecast['yhat'][train_end_index:len(z1)])
        pred=pred.reset_index()
        pred_df=pd.merge(test_df,pred,on='ds',how='left')
        pred_df.dropna(inplace=True)
        
        df=pd.DataFrame()
    
    
        
        if(len(pred_df)>0):
            
            pred_df['error_test']=pred_df.y-pred_df.yhat
        
            
        
            MSE=mse(pred_df.y,pred_df.yhat)
            RMSE=math.sqrt(MSE)
            pred_df['APE']=abs(pred_df.error_test*100/pred_df.y)
            MAPE=pred_df.APE.mean()
            min_error_rate = pred_df['APE'].quantile(0)/100
            max_error_rate = pred_df['APE'].quantile(1)/100
            median_error_rate = pred_df['APE'].quantile(.50)/100
            print("App name:",app_name)
            #print("MSE  :",MSE)
            print("RMSE :",RMSE)
            print("MAPE :",MAPE)
            
           
            mape_q98=pred_df['APE'][pred_df.APE<pred_df['APE'].quantile(0.98)].mean()
            std_MAPE = math.sqrt(((pred_df.APE-MAPE)**2).mean())
    
            df = pd.DataFrame({'length':len(z1),
                                 'test_rmse':RMSE,
                                 'test_mape':MAPE,
                                 'std_mape':std_MAPE, #standerd deviation of mape
                                 'min_error_rate':min_error_rate ,
                                 'max_error_rate':max_error_rate ,
                                 'median_error_rate':median_error_rate,
                     
                     'test_mape_98':mape_q98},
                       
                              index=[app_name])

    return(df,model,forecast,pred_df,pred_r)


### Function to select a combination , data preperation

In [7]:
#-- Function to select a combination for the run

def forcomb(s,a,df,ftime1):
    
    df2 = df[ (df.source == s)]
   
    prophet_df = pd.DataFrame()
    prophet_analysis_df = pd.DataFrame()
    prophet_future_df = pd.DataFrame()

    df2['date'] = df2.index.date
    
    df2 = pd.DataFrame(df2.groupby(by='date').app_rsp_time.max())
    
    df2 = df2.reset_index()
    df2 = df2.sort_values(by='date',ascending=True)
    df2.index = df2['date']
    del df2['date']
    df2['application'] = df.application[0]
    df2['source'] = s

    print('length of data = ',len(df2))
   
    if(len(df2)>config.limit):
             
        prophet_analysis_df,ew_model,ew_forcast,prophet_df,prophet_future_df =(create_prophet_m(a,df2,config.delay))


        
        t2 = datetime.now()
        prophet_analysis_df['total_run_time'] = round(((t2-ftime1).seconds/60),2)
        
        prophet_analysis_df['application'] = a
        prophet_analysis_df['source'] = s
        
            
       
        prophet_future_df['application'] = a
        prophet_future_df['source'] = s
        
        prophet_df['application'] = a
        prophet_df['source'] = s

    df2 = df2.reset_index()
    return prophet_df, prophet_analysis_df, prophet_future_df , df2

In [8]:


a = ap_list[0]
s = s_list[0]
prophet_analysis_df_full = pd.DataFrame()

for a in tqdm(ap_list):
    for s in s_list:
        qt1 = datetime.now()
        data = df[(df.application == a ) & (df.source==s)]

        df_t = data.registerTempTable('dummy')
        df_t = sqlContext.sql('select avg(app_rsp_time) as app_rsp_time, time_stamp, source , application  from dummy group by source, application, time_stamp')
       
        # data cleaning
        df_t = df_t[df_t.app_rsp_time!=0]
        app_rsp_time_df=df_t.toPandas()
    

        #s_array = app_rsp_time_df.source.unique()

        app_rsp_time_df = app_rsp_time_df.sort_values(by='app_rsp_time',ascending=True)       
        dates_outlook = pd.to_datetime(pd.Series(app_rsp_time_df.time_stamp),unit='ms')
        app_rsp_time_df.index = dates_outlook   
        app_rsp_time_df = app_rsp_time_df.sort_values(by='time_stamp')
        prophet_df,prophet_analysis_df,prophet_future_df,app_rsp_time_full_df = forcomb(s,a,app_rsp_time_df,qt1)

        prophet_analysis_df_full = prophet_analysis_df_full.append(prophet_analysis_df)


  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/pyspark/sql/dataframe.py:138: DeprecationWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  "Deprecated in 2.0, use createOrReplaceTempView instead.", DeprecationWarning)
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


length of data =  83


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


App name: DNS
RMSE : 14347.067542219651
MAPE : 9.901297640116233


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

App name: Kerberos
RMSE : 31921.432920459934
MAPE : 21.67788254492486


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


length of data =  91


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
100%|██████████| 5/5 [03:32<00:00, 42.79s/it]

App name: Kerberos
RMSE : 47893.9651024127
MAPE : 82.13305389834194


In [9]:
prophet_analysis_df_full

,length,test_rmse,test_mape,std_mape,min_error_rate,max_error_rate,median_error_rate,test_mape_98,total_run_time,application,source
DNS,83,14347.067542,9.901298,13.901434,0.001385,0.543710,0.047777,8.122511,0.42,DNS,10.6.1.101
DNS,91,85111.159532,167.166154,251.379000,0.545330,15.012395,1.094283,121.163624,0.38,DNS,134.141.5.104
DHCP,83,4679.961229,19.713460,14.130677,0.004188,0.521598,0.171683,18.415608,0.33,DHCP,10.6.1.101
DHCP,91,35847.273605,213.045084,319.413125,0.143294,18.019294,1.133730,158.255971,0.35,DHCP,134.141.5.104
Radius,83,215411.140954,218.314205,255.396785,0.018991,8.441537,0.811838,193.280626,0.32,Radius,10.6.1.101
Radius,91,297551.948238,453.165254,601.961855,0.007443,20.482787,1.646370,398.161342,0.33,Radius,134.141.5.104
LDAP,83,17253.779140,19.043158,12.934330,0.010976,0.571972,0.153255,17.516996,0.33,LDAP,10.6.1.101
LDAP,91,3542.271013,16.693480,30.160869,0.004622,1.764835,0.115175,11.183479,0.33,LDAP,134.141.5.104
Kerberos,83,31921.432920,21.677883,15.451168,0.007299,0.501217,0.151835,20.540130,0.33,Kerberos,10.6.1.101
Kerberos,91,47893.965102,82.133054,42.150376,0.263044,2.754134,0.811270,75.468215,0.33,Kerberos,134.141.5.104


## XGBoost Model

XGBoost is an implementation of gradient boosted decision trees designed for speed and performance that is dominative competitive machine learning.

delay_pred function is used to make history date with specified delay.

In [10]:
def delay_pred(dataset,delay):
    dataset2 = dataset.copy()
    colnames = (dataset.columns)
    for i in range((delay),len(dataset)):
        
        for j in range(0,len(colnames)):
            colmn1 = colnames[j]
            if(colmn1 in ['app_rsp_time','hour','weekday','app_rsp_time_t-1']):
                continue
            dataset2[colmn1][i] = dataset.iloc[(i-delay):i,j].mean()
            #dataset2.set_values(i,colmn1, dataset2.iloc[i:(i+delay),j].mean())
    
    return dataset2


forcast_shifter funtion is used to shift the output(app_rsp_time) as per the point to be predicted using the history data , and also it adjust the length of given x & y data after the shifting operation to avoid the na values.

In [11]:
def forcast_shifter(X,y,forcast_lag):
    
    y1=y.shift(-forcast_lag)
    X1=X.iloc[0:X.shape[0]-forcast_lag,:].astype('float64')
    y1 = y1[0:len(y1)-forcast_lag]
    return X1,y1

In [12]:
def  xgb_all(data_l_ew_g,history_lag=5,forcast_lag=1):
    
    data_l_ew_g1 = data_l_ew_g.copy()
    data_l_ew_history = delay_pred(data_l_ew_g1,history_lag)
    # Removing the outlier
    #data_l_ew_history.loc[data_l_ew_history['app_rsp_time']==max(data_l_ew_history['app_rsp_time']),'app_rsp_time']= data_l_ew_history['app_rsp_time'].quantile(.9)*.2

    ###### Removing the outlier ######
    #cutter = data_l_ew_history['app_rsp_time'].quantile(.85)
    #data_l_ew_history.loc[data_l_ew_history['app_rsp_time']> cutter,'app_rsp_time']= cutter
    #data_l_ew_history['app_rsp_time'] = np.log(data_l_ew_history['app_rsp_time'])
    
    import random
    random.seed(100)
    data_l_ew_history['date'] = data_l_ew_history.index
    data_l_ew_history=data_l_ew_history.sort_values(by='date')
    from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
    cal = calendar()
    holidays = cal.holidays(start = data_l_ew_history.date.min(), end = data_l_ew_history.date.max())
    #data_l_ew_history["IS_HOLIDAY"] = data_l_ew_history.date.isin(holidays)
    #data_l_ew_history["IS_WORKDAY"] = data_l_ew_history.date.apply(lambda x: 0 if x.dayofweek > 5 else 1  )
    import xgboost as xgb
    from sklearn.cross_validation import train_test_split as ttsplit
    from sklearn.datasets import load_boston
    from sklearn.metrics import mean_squared_error as mse

    #--- Model without is_workday and is_holiday ---#
    import numpy as np
    #X = pd.concat([data_l_ew_history.iloc[:,0:13],data_l_ew_history.iloc[:,14:16]],axis=1).values
    X = data_l_ew_history.iloc[0:len(data_l_ew_history),1:data_l_ew_history.shape[1]-1].values
    y = np.array((data_l_ew_history['app_rsp_time']))
    
    #--- Shifting as per required forcast ------
    X1,y1 = forcast_shifter(data_l_ew_history.iloc[0:len(data_l_ew_history),1:data_l_ew_history.shape[1]-1],
                            data_l_ew_history['app_rsp_time'],forcast_lag)
    X = X1.values
    #y = np.array(np.log(y1))
    y = np.array(y1)
    
    # split data into training and testing sets
    # then split training set in half
    #X_train, X_test, y_train, y_test = ttsplit(X, y, test_size=0.3, random_state=0)
    #split2 = int(len(X)*.7)
    split2 = int(len(X)-24)
    X_train = X[0:split2,:]
    X_test = X[split2:len(X),:]
    y_train = y[0:split2]
    y_test = y[split2:len(y)]
    xg_train_1 = xgb.DMatrix(X_train, label=y_train)
    #xg_train_2 = xgb.DMatrix(X_train_2, label=y_train_2)
    xg_test = xgb.DMatrix(X_test, label=y_test)
    import datetime
    s_time = datetime.datetime.now()

    params = {'objective': 'reg:linear', 'verbose': True , 'eval' : 'rmse ' ,'gamma':.01,
             'max_depth':3, 'min_child_weight':3,'eta':.02,'num_round':350}
    #params = {  'objective':'reg:linear' , 'verbose': True , 'eval' : 'rmse ' }
    model_1_mean5 = xgb.train(params, xg_train_1, 200)
    e_time = datetime.datetime.now()
    run_time = (e_time - s_time)
    print((run_time.seconds)/60,"minuts")

    #--- Training and testing rmse finding ---#
    
    predicted1 = model_1_mean5.predict(xg_train_1)
    
    #y_train = np.exp(y_train)
    #predicted1 = np.exp(predicted1)
    
    delta = pd.Series(( y_train - predicted1))
    compar_train = pd.DataFrame({'predicted':predicted1,'actual':y_train,'error':delta})
    #compar_train['perc']=compar_train['error']/compar_train['actual']*100

    mse_train =(mse(compar_train['predicted'] , compar_train['actual']))
    import math
    xgb_rmse_train = math.sqrt(mse_train)
    print('training rmse =',round(xgb_rmse_train,2))
    compar_train['perc_error']= abs(compar_train['error']/compar_train['actual']*100)
    xgb_mape_train = (compar_train.perc_error.mean())
    print('MAPE = ',xgb_mape_train)
    
    q98=compar_train['perc_error'].quantile(0.98)
    mape_q98=compar_train['perc_error'][compar_train.perc_error<compar_train['perc_error'].quantile(0.98)].mean()
    print('MAPE clipped at 98% = ',mape_q98)
    
    
    ### Test set manipulations ###
    predicted1 = model_1_mean5.predict(xg_test)
    
    #y_test = np.exp(y_test)
    #predicted1 = np.exp(predicted1)
    
    delta = pd.Series(( y_test - predicted1))
    compar_test = pd.DataFrame({'predicted':predicted1,'actual':y_test,'error':delta})
    compar_test['perc_error']= abs(compar_test['error']/compar_test['actual']*100)
    xgb_mse_test =(mse(compar_test['predicted'] , compar_test['actual']))
    xgb_rmse_test = math.sqrt(xgb_mse_test)

    print('test rmse',xgb_rmse_test )
    
    xgb_mape_test = compar_test.perc_error.mean()
    print('MAPE = ',xgb_mape_test)
    
    q98=compar_test['perc_error'].quantile(0.98)
    mape_q98_test =compar_test['perc_error'][compar_test.perc_error<compar_test['perc_error'].quantile(0.98)].mean()
    print('MAPE clipped at 98% = ',mape_q98_test)
    
    mm ='xgboost_hist'+str(history_lag)+'_forcast_'+str(forcast_lag)
    df = pd.DataFrame({'Model':[mm],#'predicted_t':[forcast_lag],
                         'train_rmse':[round(xgb_rmse_train,2)],
                         'test_rmse':[round(xgb_rmse_test,2)],
                         'train_mape':[round(xgb_mape_train,2)],
                         'test_mape':[round(xgb_mape_test,2)],
                        'train_mape_98':[round(mape_q98,2)],
                       'test_mape_98':[round(mape_q98_test,2)]
                      })
    return df,compar_train,compar_test

In [20]:


a = ap_list[0]
s = s_list[0]
prophet_analysis_df_full = pd.DataFrame()

for a in tqdm(ap_list):
    for s in s_list:
        qt1 = datetime.now()
        data = df[(df.application == a ) & (df.source==s)]

        df_t = data.registerTempTable('dummy')
        df_t = sqlContext.sql('select avg(app_rsp_time) as app_rsp_time, time_stamp, source , application  from dummy group by source, application, time_stamp')
       
        # data cleaning
        df_t = df_t[df_t.app_rsp_time!=0]
        app_rsp_time_df=df_t.toPandas()
    

        #s_array = app_rsp_time_df.source.unique()

        app_rsp_time_df = app_rsp_time_df.sort_values(by='app_rsp_time',ascending=True)       
        dates_outlook = pd.to_datetime(pd.Series(app_rsp_time_df.time_stamp),unit='ms')
        app_rsp_time_df.index = dates_outlook   
        app_rsp_time_df = app_rsp_time_df.sort_values(by='time_stamp')

        app_rsp_time_df['date'] = app_rsp_time_df.index.date
        app_rsp_time_df = pd.DataFrame(app_rsp_time_df.groupby(by='date').app_rsp_time.max())
        app_rsp_time_df = app_rsp_time_df.reset_index()
        app_rsp_time_df = app_rsp_time_df.sort_values(by='date',ascending=True)
        app_rsp_time_df.index = app_rsp_time_df['date']
        del app_rsp_time_df['date']


  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/pyspark/sql/dataframe.py:138: DeprecationWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  "Deprecated in 2.0, use createOrReplaceTempView instead.", DeprecationWarning)


length of data =  1874
App name: DNS
RMSE : 9767.66357986765
MAPE : 21.822413202068496
length of data =  1930


 20%|██        | 1/5 [00:55<03:42, 55.59s/it]

App name: DNS
RMSE : 12688.36443022666
MAPE : 92.61242148594886
length of data =  1874
App name: DHCP
RMSE : 3015.662779071946
MAPE : 31.849032037639372
length of data =  1930


 40%|████      | 2/5 [01:46<02:42, 54.13s/it]

...

App name: Radius
RMSE : 74574.22757211838
MAPE : 82.73393226864061
length of data =  1874
App name: LDAP
RMSE : 13244.369473434286
MAPE : 15.53367889376674
length of data =  1930


 80%|████████  | 4/5 [03:33<00:53, 53.93s/it]

App name: LDAP
RMSE : 1848.537421506424
MAPE : 15.437597912825971
length of data =  1874
App name: Kerberos
RMSE : 21283.111310100852
MAPE : 23.155300507321623
length of data =  1930


100%|██████████| 5/5 [04:27<00:00, 54.21s/it]

App name: Kerberos
RMSE : 9785.607409772696
MAPE : 26.03856262297717


In [20]:
data.head()

Row(time_stamp=1510801200000, source='134.141.5.104', zone='\\N', target_address='134.141.22.168/', application='Kerberos', applicationgroup='VPN and Security', target_attribute_id=215904035, byte_count=18656, tx_byte_count=8388, rx_byte_count=10268, flow_count=9, tx_flow_count=4, rx_flow_count=5, tcp_rsp_time=987, app_rsp_time=3048, user_data='\\N')

In [22]:
df_t = data.registerTempTable('dummy')
df_t = sqlContext.sql('select avg(app_rsp_time) as app_rsp_time, time_stamp, avg(byte_count)  as byte_count, avg(flow_count) as flow_count , avg(rx_byte_count) as rx_byte_count , avg(rx_flow_count) as rx_flow_count , avg(tcp_rsp_time) as tcp_rsp_time , avg(tx_byte_count) as tx_byte_count , avg(tx_flow_count) as tx_flow_count from dummy group by source, application, time_stamp')


/opt/conda/lib/python3.6/site-packages/pyspark/sql/dataframe.py:138: DeprecationWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  "Deprecated in 2.0, use createOrReplaceTempView instead.", DeprecationWarning)


In [29]:
app_rsp_time_df

,app_rsp_time,time_stamp,byte_count,flow_count,rx_byte_count,rx_flow_count,tcp_rsp_time,tx_byte_count,tx_flow_count
time_stamp,,,,,,,,,
2017-11-11 00:00:00,43776.264957,1510358400000,275658.149573,181.756410,142102.034188,90.905983,13277.290598,133556.115385,90.850427
2017-11-11 01:00:00,39715.868778,1510362000000,295108.936652,192.701357,152100.140271,96.339367,11406.131222,143008.796380,96.361991
2017-11-11 02:00:00,48935.561321,1510365600000,281056.523585,189.500000,145160.311321,94.900943,14406.292453,135896.212264,94.599057
2017-11-11 03:00:00,43377.799043,1510369200000,294189.444976,198.717703,151354.019139,99.306220,13603.095694,142835.425837,99.411483
2017-11-11 04:00:00,37502.680412,1510372800000,765042.634021,421.958763,387375.752577,210.969072,14723.231959,377666.881443,210.989691
2017-11-11 05:00:00,42095.516129,1510376400000,309150.080645,201.876344,159800.580645,100.924731,14894.360215,149349.500000,100.951613
2017-11-11 06:00:00,38777.823529,1510380000000,313159.967914,207.385027,161911.235294,103.582888,15558.379679,151248.732620,103.802139
2017-11-11 07:00:00,41516.089947,1510383600000,322182.968254,201.957672,157265.735450,100.978836,15437.137566,164917.232804,100.978836
2017-11-11 08:00:00,46376.103093,1510387200000,301581.010309,197.386598,155777.938144,98.664948,15900.556701,145803.072165,98.721649


In [30]:
del app_rsp_time_df['time_stamp']

In [25]:
df_t = data.registerTempTable('dummy')
df_t = sqlContext.sql('select avg(app_rsp_time) as app_rsp_time, time_stamp, avg(byte_count)  as byte_count, avg(flow_count) as flow_count , avg(rx_byte_count) as rx_byte_count , avg(rx_flow_count) as rx_flow_count , avg(tcp_rsp_time) as tcp_rsp_time , avg(tx_byte_count) as tx_byte_count , avg(tx_flow_count) as tx_flow_count from dummy group by source, application, time_stamp')


# data cleaning
df_t = df_t[df_t.app_rsp_time!=0]
app_rsp_time_df=df_t.toPandas() 

   
app_rsp_time_df = app_rsp_time_df.sort_values(by='app_rsp_time',ascending=True)       
dates_outlook = pd.to_datetime(pd.Series(app_rsp_time_df.time_stamp),unit='ms')
app_rsp_time_df.index = dates_outlook   
app_rsp_time_df = app_rsp_time_df.sort_values(by='time_stamp')

app_rsp_time_df['date'] = app_rsp_time_df.index.date

/opt/conda/lib/python3.6/site-packages/pyspark/sql/dataframe.py:138: DeprecationWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  "Deprecated in 2.0, use createOrReplaceTempView instead.", DeprecationWarning)


In [28]:
#app_rsp_time_df = pd.DataFrame(app_rsp_time_df.groupby(by='date').app_rsp_time.max())
app_rsp_time_df = app_rsp_time_df.reset_index()
app_rsp_time_df = app_rsp_time_df.sort_values(by='date',ascending=True)
app_rsp_time_df.index = app_rsp_time_df['date']
del app_rsp_time_df['date']

weekday = app_rsp_time_df['time_stamp'].dt.weekday
app_rsp_time_df['weekday'] = weekday

ValueError: cannot insert time_stamp, already exists

In [15]:
del app_rsp_time_df['time_stamp']

In [33]:
df,compare_train,compare_test=xgb_all(app_rsp_time_df,5,0)
df

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.0 minuts
training rmse = 6327.22
MAPE =  13.631411005866841
MAPE clipped at 98% =  11.595851390983789
test rmse 6961.8193387191395
MAPE =  11.555158949428831
MAPE clipped at 98% =  10.668879331764773


,Model,train_rmse,test_rmse,train_mape,test_mape,train_mape_98,test_mape_98
0,xgboost_hist5_forcast_0,6327.22,6961.82,13.63,11.56,11.6,10.67
